In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

***vis_videos_seen_skipped* function**:
* Input: players_info dictionary
* Displays bar chart showing for each video the total number of times it has been seen and skipped

In [2]:
outputVideo=widgets.Output()

def vis_videos_seen_skipped():
    vis_videos_seen_skipped=interactive(construct_graph_video,
                        #constructing a dropdown to select how many data we want to see in visualisations for x axis
                       xaxis=widgets.Dropdown(options=[('All datas',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
                                description='x axe',
                                disabled=False),
                        #constructing a dropdown to order for alphabetic keys or for values
                        order=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                                description='Order',
                                disabled=False),
                       percentageVideoSkipped=False,
                       orderLabelValues=[("Order by all seen and skipped videos values",0),
                                         ("Order by seens videos values","seen"),
                                         ("Order by skipped videos values","skipped")])
    md_text = widgets.HTML("<h4>Bar chart showing for each video the total number of times it has been seen and skipped</h4>")
    with outputVideo:
        print("Updating visualisation with the current selection of players and videos...")
    return VBox([md_text, vis_videos_seen_skipped, outputVideo])

def construct_graph_video(percentageVideoSkipped, xaxis,order, orderLabelValues):
    #get all players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    videosSelected=checkboxesVideoSelected.get_selected_options()
    videos_seen = {}
    videos_skipped = {}

    for player in playersSelected:
        # videos seen
        for video_seen in players_info[player]["videos_seen"]:
            if video_seen in videosSelected:
                if video_seen in videos_seen.keys():
                    videos_seen[video_seen] += 1
                else:
                    videos_seen[video_seen] = 1
                if video_seen not in videos_skipped.keys():
                    videos_skipped[video_seen] = 0
        # videos skipped
        for video_skipped in players_info[player]["videos_skipped"]:
            if video_skipped in videosSelected:
                if video_skipped in videos_skipped.keys():
                    videos_skipped[video_skipped] += 1
                else:
                    videos_skipped[video_skipped] = 1
                if video_skipped not in videos_seen.keys():
                    videos_seen[video_skipped] = 0
                else:
                    videos_seen[video_skipped] -= 1
    videosseenskipped={}
    for video in videosSelected:
        videosseenskipped[video]={"seen":videos_seen[video],"skipped":videos_skipped[video]}
    if orderLabelValues==0:
        lambdafunc=lambda item:item[1]["seen"]+item[1]["skipped"]
    else:
        lambdafunc=lambda item:item[1][orderLabelValues]
    videosseenskipped = orderDict(videosseenskipped,order,lambdafunc)
    nbgraphshown, videosseenskipped,N = organizeDictPerxAxisSelected(videos_seen,xaxis, videosseenskipped)
    # Bar chart showing for each video
    # the total number of times it has been seen and skipped
    for n in range(nbgraphshown):
        v_seen=[videosseenskipped[n][video]["seen"] for video in videosseenskipped[n].keys()]
        v_skipped=[videosseenskipped[n][video]["skipped"] for video in videosseenskipped[n].keys()]
        v_list=[video for video in videosseenskipped[n].keys()]
        if percentageVideoSkipped:
            if N>0:
                get_percentage_bar(v_skipped, v_seen, v_list)
            else: 
                noDataToFillVis(10)
            plt.ylabel("Percentage of videos skipped")
        else:
            if N>0:
                if n<nbgraphshown-1:
                    ind = np.arange(N)
                else:
                    ind= np.arange(len(v_list))
                get_two_superposed_values_bars(ind,v_list,
                                            v_seen, "Seen",'orange',
                                            v_skipped, "Seen but skipped",'lightblue')
            else:
                noDataToFillVis(10)
            plt.ylabel("Number of times")
        plt.xticks(rotation=45, ha="right")
        plt.xlabel("Video")
        plt.title("Videos seen and skipped by players")
        plt.show()
    with outputVideo:
        clear_output(wait=False)